In [174]:
import requests
import re
from bs4 import BeautifulSoup 

In [175]:
def pridej_rocnik(adresa):
    
    response = requests.get(adresa)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    rocnik = []
    
    try:
        letapane = re.search("\d\d\d\d", soup.find(class_="content").text).group(0)
    except:
        pass
    
    try:
        datum = re.search("\d{1,2}\.\s{0,2}\w*\s{1,2}\d{4}", soup.find(class_="content").text).group(0)
    except:
        datum = None
    
    divs = soup.find_all('div', {'data-groupnames': ['0', '1']})

    kategorie = ""
    
    for div in divs:
        
        kategorie = div.text.strip()
        
        next_div = div.find_next('div')
        
        while next_div and 'data-groupnames' not in next_div.attrs:
        
            if next_div.find("a"):

                odkazy = next_div.find_all("a")
                
                for o in odkazy:
                    if 'creator' in o['href']:
    
                        radek = {}
                        
                        radek['rok'] = letapane
                        radek['datum'] = datum
                        radek['kategorie'] = kategorie
                        
                        try:
                            radek['kdo'] = f"""{o.text.strip()} ({o['href'].split("csfdid=")[-1]})"""
                        except:
                            radek['kdo'] = o.text.strip()
                
                        radek['film'] = None
                
                        for d in odkazy:
                            if 'movie=' in d['href']:
                                radek['film'] = d.text.strip()
                                            
                        radek['co'] = 'nominace'
                        if 'DRŽITEL' in next_div.text:
                            radek['co'] = 'vyhra'

                        rocnik.append(radek)

            next_div = next_div.find_next('div')
            
    rocnik_cisty = []
    for dictionary in rocnik:
        if dictionary not in rocnik_cisty:
            rocnik_cisty.append(dictionary)
            
    return rocnik_cisty

In [176]:
pridej_rocnik("https://www.filmovaakademie.cz/cz/1993/nominace")

[{'rok': '1993',
  'datum': '25. února 1994',
  'kategorie': 'Nejlepší film',
  'kdo': 'Jiří Ježek (132937)',
  'film': 'Šakalí léta',
  'co': 'vyhra'},
 {'rok': '1993',
  'datum': '25. února 1994',
  'kategorie': 'Nejlepší režie',
  'kdo': 'Jan Hřebejk (3195)',
  'film': 'Šakalí léta',
  'co': 'vyhra'},
 {'rok': '1993',
  'datum': '25. února 1994',
  'kategorie': 'Hlavní ženský herecký výkon',
  'kdo': 'Jiřina Bohdalová (1088)',
  'film': 'Nesmrtelná teta',
  'co': 'vyhra'},
 {'rok': '1993',
  'datum': '25. února 1994',
  'kategorie': 'Hlavní mužský herecký výkon',
  'kdo': 'Josef Abrhám (931)',
  'film': 'Šakalí léta',
  'co': 'vyhra'},
 {'rok': '1993',
  'datum': '25. února 1994',
  'kategorie': 'Nejlepší scénář',
  'kdo': 'Václav Šašek (40793)',
  'film': 'Helimadoe',
  'co': 'vyhra'},
 {'rok': '1993',
  'datum': '25. února 1994',
  'kategorie': 'Nejlepší kamera',
  'kdo': 'Jaroslav Brabec (3156)',
  'film': 'Krvavý román',
  'co': 'vyhra'},
 {'rok': '1993',
  'datum': '25. února 1

In [177]:
roky = range(1993,2025)

In [178]:
import pandas as pd

In [179]:
df = pd.DataFrame()
for r in roky:
    print(r)
    df = pd.concat([df, pd.DataFrame(pridej_rocnik(f"https://www.filmovaakademie.cz/cz/{r}/nominace"))])
    df = pd.concat([df, pd.DataFrame(pridej_rocnik(f"https://www.filmovaakademie.cz/cz/{r}/no"))])
    df = pd.concat([df, pd.DataFrame(pridej_rocnik(f"https://www.filmovaakademie.cz/cz/{r}/nominace-filmy"))])
    df = pd.concat([df, pd.DataFrame(pridej_rocnik(f"https://www.filmovaakademie.cz/cz/{r}/nominace-televizni-projekty"))])
    df = pd.concat([df, pd.DataFrame(pridej_rocnik(f"https://www.filmovaakademie.cz/cz/{r}/studentske-filmy"))])
df

1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


,rok,datum,kategorie,kdo,film,co
0,1993,25. února 1994,Nejlepší film,Jiří Ježek (132937),Šakalí léta,vyhra
1,1993,25. února 1994,Nejlepší režie,Jan Hřebejk (3195),Šakalí léta,vyhra
2,1993,25. února 1994,Hlavní ženský herecký výkon,Jiřina Bohdalová (1088),Nesmrtelná teta,vyhra
3,1993,25. února 1994,Hlavní mužský herecký výkon,Josef Abrhám (931),Šakalí léta,vyhra
4,1993,25. února 1994,Nejlepší scénář,Václav Šašek (40793),Helimadoe,vyhra
...,...,...,...,...,...,...
137,2023,9. března 2024,Cena Magnesia za nejlepší studentský film,Anna Wowra (249417),Jsme v tom spolu,nominace
138,2023,9. března 2024,Cena Magnesia za nejlepší studentský film,Petr Pylypčuk (578198),Osmý den,nominace
139,2023,9. března 2024,Nejlepší filmový plakát,Zuzana Burgrová (/cz/detail?creator=Zuzana Bur...,Němá tajemství,vyhra
140,2023,9. března 2024,Nejlepší filmový plakát,Robert V. Novák (/cz/detail?creator=Robert V. ...,Němá tajemství,vyhra


In [180]:
df = df.reset_index()

In [181]:
import os

In [182]:
df['datum'].drop_duplicates()

0        25. února 1994
10       3. března 1995
89       2. března 1996
135       1.března 1997
189      28. února 1998
236      28. února 1999
290     4. března  2000
343      3. března 2001
396      2. března 2002
452      1. března 2003
511      3. března 2004
562      5. března 2005
610      25. února 2006
656      3. března 2007
710      1. března 2008
763      7. března 2009
809      6. března 2010
862      5. března 2011
954      3. března 2012
1064     2. března 2013
1185     22. února 2014
1274     21. února 2015
1369     5. března 2016
1471     4. března 2017
1588    10. března 2018
1703    23. března 2019
1823     7. března 2020
1931     6. března 2021
2074     5. března 2022
2222     4. března 2023
2377     9. března 2024
Name: datum, dtype: object

In [183]:
def zdatuj(datum):
    rok = int(datum.split(" ")[-1])
    den = int(datum.split(".")[0])
    m = datum.split(".")[1].strip().split(" ")[0].strip()
    if m == 'února':
        mesic = 2
    if m == 'března':
        mesic = 3
    return f"{rok}-{mesic:02}-{den:02}"

In [184]:
zdatuj("9. března 2024")

'2024-03-09'

In [185]:
df

,index,rok,datum,kategorie,kdo,film,co
0,0,1993,25. února 1994,Nejlepší film,Jiří Ježek (132937),Šakalí léta,vyhra
1,1,1993,25. února 1994,Nejlepší režie,Jan Hřebejk (3195),Šakalí léta,vyhra
2,2,1993,25. února 1994,Hlavní ženský herecký výkon,Jiřina Bohdalová (1088),Nesmrtelná teta,vyhra
3,3,1993,25. února 1994,Hlavní mužský herecký výkon,Josef Abrhám (931),Šakalí léta,vyhra
4,4,1993,25. února 1994,Nejlepší scénář,Václav Šašek (40793),Helimadoe,vyhra
...,...,...,...,...,...,...,...
2514,137,2023,9. března 2024,Cena Magnesia za nejlepší studentský film,Anna Wowra (249417),Jsme v tom spolu,nominace
2515,138,2023,9. března 2024,Cena Magnesia za nejlepší studentský film,Petr Pylypčuk (578198),Osmý den,nominace
2516,139,2023,9. března 2024,Nejlepší filmový plakát,Zuzana Burgrová (/cz/detail?creator=Zuzana Bur...,Němá tajemství,vyhra
2517,140,2023,9. března 2024,Nejlepší filmový plakát,Robert V. Novák (/cz/detail?creator=Robert V. ...,Němá tajemství,vyhra


In [186]:
df_copy = df.copy()
df_copy['datum'] = df_copy['datum'].apply(lambda x: zdatuj(x))

In [188]:
df_copy

,index,rok,datum,kategorie,kdo,film,co
0,0,1993,1994-02-25,Nejlepší film,Jiří Ježek (132937),Šakalí léta,vyhra
1,1,1993,1994-02-25,Nejlepší režie,Jan Hřebejk (3195),Šakalí léta,vyhra
2,2,1993,1994-02-25,Hlavní ženský herecký výkon,Jiřina Bohdalová (1088),Nesmrtelná teta,vyhra
3,3,1993,1994-02-25,Hlavní mužský herecký výkon,Josef Abrhám (931),Šakalí léta,vyhra
4,4,1993,1994-02-25,Nejlepší scénář,Václav Šašek (40793),Helimadoe,vyhra
...,...,...,...,...,...,...,...
2514,137,2023,2024-03-09,Cena Magnesia za nejlepší studentský film,Anna Wowra (249417),Jsme v tom spolu,nominace
2515,138,2023,2024-03-09,Cena Magnesia za nejlepší studentský film,Petr Pylypčuk (578198),Osmý den,nominace
2516,139,2023,2024-03-09,Nejlepší filmový plakát,Zuzana Burgrová (/cz/detail?creator=Zuzana Bur...,Němá tajemství,vyhra
2517,140,2023,2024-03-09,Nejlepší filmový plakát,Robert V. Novák (/cz/detail?creator=Robert V. ...,Němá tajemství,vyhra


In [189]:
df_copy.to_json(os.path.join('data','ceske_lvy_raw.json'))